# Create filtered antibody escape CSVs based on configuration for plotting

This notebook filters the antibody escape data based on filters applied when plotting the data and outputs filtered escape CSV. In addition, escape profiles and logoplots are created for each antibody selection.

In [ ]:
# Imports 
import os
import warnings
import dmslogo
import pandas as pd
import seaborn as sns
import matplotlib.colors
import matplotlib.pyplot as plt

# Plotting colors
tol_muted_adjusted = [
    "#000000",
    "#CC6677", 
    "#1f78b4", 
    "#DDCC77", 
    "#117733", 
    "#882255", 
    "#88CCEE",
    "#44AA99", 
    "#999933", 
    "#AA4499", 
    "#DDDDDD",
]

# Create color palette
def color_gradient_hex(start, end, n):
    """Color function from polyclonal"""
    cmap = matplotlib.colors.LinearSegmentedColormap.from_list(
            name="_", colors=[start, end], N=n
        )
    return [matplotlib.colors.rgb2hex(tup) for tup in cmap(list(range(0, n)))]

# Seaborn style settings
sns.set(rc={"figure.dpi":300, "savefig.dpi":300})
sns.set_style("ticks")
sns.set_palette(tol_muted_adjusted)

# Suppress warnings
warnings.simplefilter("ignore")

File paths for data:

In [ ]:
# this cell is tagged as `parameters` for papermill parameterization
func_scores = None

escape_377H = None
escape_89F = None
escape_2510C = None
escape_121F = None
escape_256A = None
escape_372D = None

min_times_seen = None
min_func_score = None
n_selections = None
frac_models = None

out_dir = None
out_dir_images = None

filtered_escape_377H = None
filtered_escape_89F = None
filtered_escape_2510C = None
filtered_escape_121F = None
filtered_escape_256A = None
filtered_escape_372D = None

contacts_89F = None
contacts_377H = None
contacts_256A = None
contacts_2510C = None
contacts_121F = None
contacts_372D = None

scale_bar_89F = None
scale_bar_377H = None
scale_bar_256A = None
scale_bar_2510C = None
scale_bar_121F = None
scale_bar_372D = None

saved_image_path = None
validation_image_path = None

In [ ]:
# # Uncomment for running interactive
# func_scores = "../results/func_effects/averages/293T_entry_func_effects.csv"

# escape_377H = "../results/antibody_escape/averages/377H_mut_effect.csv"
# escape_89F = "../results/antibody_escape/averages/89F_mut_effect.csv"
# escape_2510C = "../results/antibody_escape/averages/2510C_mut_effect.csv"
# escape_121F = "../results/antibody_escape/averages/121F_mut_effect.csv"
# escape_256A = "../results/antibody_escape/averages/256A_mut_effect.csv"
# escape_372D = "../results/antibody_escape/averages/372D_mut_effect.csv"

# min_times_seen = 2
# min_func_score = -1.5
# n_selections = 8
# frac_models = 1

# out_dir = "../results/filtered_antibody_escape_CSVs/"
# out_dir_images = "../results/antibody_escape_profiles/"

# filtered_escape_377H = "../results/filtered_antibody_escape_CSVs/377H_filtered_mut_effect.csv"
# filtered_escape_89F = "../results/filtered_antibody_escape_CSVs/89F_filtered_mut_effect.csv"
# filtered_escape_2510C = "../results/filtered_antibody_escape_CSVs/2510C_filtered_mut_effect.csv"
# filtered_escape_121F = "../results/filtered_antibody_escape_CSVs/121F_filtered_mut_effect.csv"
# filtered_escape_256A = "../results/filtered_antibody_escape_CSVs/256A_filtered_mut_effect.csv"
# filtered_escape_372D = "../results/filtered_antibody_escape_CSVs/372D_filtered_mut_effect.csv"

# contacts_89F = "../data/antibody_contacts/antibody_contacts_89F.csv"
# contacts_377H = "../data/antibody_contacts/antibody_contacts_377H.csv"
# contacts_256A = "../data/antibody_contacts/antibody_contacts_256A.csv"
# contacts_2510C = "../data/antibody_contacts/antibody_contacts_2510C.csv"
# contacts_121F = "../data/antibody_contacts/antibody_contacts_121F.csv"
# contacts_372D = "../data/antibody_contacts/antibody_contacts_372D.csv"

# scale_bar_89F = "../results/antibody_escape_profiles/89F_scale_bar.svg"
# scale_bar_377H = "../results/antibody_escape_profiles/377H_scale_bar.svg"
# scale_bar_256A = "../results/antibody_escape_profiles/256A_scale_bar.svg"
# scale_bar_2510C = "../results/antibody_escape_profiles/2510C_scale_bar.svg"
# scale_bar_121F = "../results/antibody_escape_profiles/121F_scale_bar.svg"
# scale_bar_372D = "../results/antibody_escape_profiles/372D_scale_bar.svg"

# saved_image_path = "../results/antibody_escape_profiles/antibody_escape_profiles.svg"
# validation_image_path = "../results/antibody_escape_profiles/validation_escape_profile.svg"

Function to create filtered CSVs of the antibody escape as well as plot escape profiles as summed site escape lineplots and logoplots that are colored by functional scores. Escape scores less than 0 are clipped to 0 and functional scores greater than 0 are clipped to 0. Functional scale bar 0 must be changed to **≥0** afterwards. Contact sites are highlighted gray in logoplots.

In [ ]:
def get_filtered_csv(
    escape_file,  
    contacts_file,
    func_scores, 
    min_times_seen, 
    min_func_score, 
    n_selections, 
    frac_models,
    line_plot,
    logo_plot,
    scale_bar_image = None,
    output_file = None,
    sites = None,
):
    """
    Function that filters and writes an antibody escape csv.
    Also creates summed escape profiles and logoplots.
    """

    antibody_name = escape_file.split("/")[-1].split("_")[0]

    # Read data
    escape_df = pd.read_csv(escape_file)
    func_scores = pd.read_csv(func_scores)
    contacts_df = pd.read_csv(contacts_file)

    # **
    # Pipeline does not filter functional scores 
    # prior to filtering antibody selections
    # **
    # Filter functional scores
    # func_scores = (
    #     func_scores.loc[
    #         (func_scores["times_seen"] >= min_times_seen)
    #         &
    #         (func_scores["n_selections"] >= n_selections)
    #     ]
    # )
    
    # Create mutation column to match antibody df
    func_scores["site"] = func_scores["site"].astype(str)
    func_scores["mutation"] = func_scores["wildtype"] + func_scores["site"] + func_scores["mutant"]
    func_scores_filter = func_scores.loc[func_scores["effect"] >= min_func_score]["mutation"].tolist()

    # Filter escape df
    escape_df = (
        escape_df.loc[
            (escape_df["mutation"].isin(func_scores_filter))
            &
            (escape_df["times_seen"] >= min_times_seen)
            &
            (escape_df["frac_models"] >= frac_models)
        ]
        .reset_index(drop=True)
    )

    # Write filtered escape to csv
    if output_file != None:
        escape_df.to_csv(output_file, index=False)

    # Clip lower scores to 0
    escape_df["escape_median"] = escape_df["escape_median"].clip(lower=0)

    # Summed escape to get top 20 escape sites to show
    summed_df = (
        escape_df
        .groupby("site")
        .aggregate({"escape_median" : "sum"})
        .rename(columns={"escape_median" : "site_escape"})
        .reset_index()
    )
    # Top escape sites for each antibody combined to show
    if sites == None:
        sites = sorted(summed_df.nlargest(20, "site_escape")["site"].tolist())
    escape_df["show_site"] = escape_df.apply(lambda x: True if x["site"] in sites else False, axis=1)
    
    # Shade contact sites in logo plot
    shade_sites = list(contacts_df.loc[contacts_df["distance"] == 4]["position"].unique())
    escape_df["shade_site"] = escape_df.apply(lambda x: "#DDDDDD" if x["site"] in shade_sites else None, axis=1)
    escape_df["shade_alpha"] = 0.15
    escape_df = (
        escape_df.merge(
            summed_df,
            how="left",
            on="site",
            validate="many_to_one",
        )
    )

    # Drop extra columns
    escape_df = (
        escape_df.drop(
            escape_df.columns.difference([
                "site", 
                "mutant", 
                "escape_median", 
                "show_site", 
                "shade_site",
                "shade_alpha",
                "site_escape", 
            ]), axis=1)
    )

    # Fill in missing sites
    seen_list = [False]*491
    for index in range(len(escape_df.index)):
        site = escape_df.at[index, "site"] - 1 
        seen_list[site] = True
    for index, seen in enumerate(seen_list):
        if seen == False:
            site = index + 1
            # Add missing sites
            escape_df.loc[len(escape_df.index)] = [
                site,
                "X",
                0,
                False,
                None,
                None,
                0,
            ]

    # Sort by site
    escape_df = (
        escape_df
        .sort_values(by="site")
        .astype({"mutant" : "str"})
        .reset_index(drop=True)
    )

    # Merge functional and escape dfs
    func_scores["site"] = func_scores["site"].astype("int")
    func_scores = func_scores.loc[func_scores["mutant"] != "*"] # remove stop codons
    func_scores["effect"] = func_scores["effect"].clip(upper=0) # clip scores greater than 0 to 0
    escape_df = (
        escape_df.merge(
            func_scores,
            how="left",
            on=["site", "mutant"],
            validate="one_to_one",
        )
    )
    escape_df["effect"] = escape_df["effect"].fillna(-4) # missing functional values are filled as -4 to make less visible
    
    # Add color column for logo plots
    if antibody_name == "2510C":
        # Create scale bar based on functional effects
        func_color_map = dmslogo.colorschemes.ValueToColorMap(
            minvalue=func_scores["effect"].min(),
            maxvalue=func_scores["effect"].max(),
            cmap=matplotlib.colors.ListedColormap(color_gradient_hex("white", "#44AA99", n=20))
        )
        escape_df = (
            escape_df.assign(
                color=lambda x: x["effect"].map(func_color_map.val_to_color)
            )
        )
        fig_scale, ax_scale = func_color_map.scale_bar(
            orientation="vertical", 
            label="effect on cell entry",
            # low_high_ticks_only=True,
        )
    if antibody_name == "121F":
        # Create scale bar based on functional effects
        func_color_map = dmslogo.colorschemes.ValueToColorMap(
            minvalue=func_scores["effect"].min(),
            maxvalue=func_scores["effect"].max(),
            cmap=matplotlib.colors.ListedColormap(color_gradient_hex("white", "#999933", n=20))
        )
        escape_df = (
            escape_df.assign(
                color=lambda x: x["effect"].map(func_color_map.val_to_color)
            )
        )
        fig_scale, ax_scale = func_color_map.scale_bar(
            orientation="vertical", 
            label="effect on cell entry",
            # low_high_ticks_only=True,
        )
    if antibody_name == "377H" or antibody_name == "256A" or antibody_name == "372D":
        # Create scale bar based on functional effects
        func_color_map = dmslogo.colorschemes.ValueToColorMap(
            minvalue=func_scores["effect"].min(),
            maxvalue=func_scores["effect"].max(),
            cmap=matplotlib.colors.ListedColormap(color_gradient_hex("white", "#AA4499", n=20))
        )
        escape_df = (
            escape_df.assign(
                color=lambda x: x["effect"].map(func_color_map.val_to_color)
            )
        )
        fig_scale, ax_scale = func_color_map.scale_bar(
            orientation="vertical", 
            label="effect on cell entry",
            # low_high_ticks_only=True,
        )
    if antibody_name == "89F":
        # Create scale bar based on functional effects
        func_color_map = dmslogo.colorschemes.ValueToColorMap(
            minvalue=func_scores["effect"].min(),
            maxvalue=func_scores["effect"].max(),
            cmap=matplotlib.colors.ListedColormap(color_gradient_hex("white", "#117733", n=20))
        )
        escape_df = (
            escape_df.assign(
                color=lambda x: x["effect"].map(func_color_map.val_to_color)
            )
        )
        fig_scale, ax_scale = func_color_map.scale_bar(
            orientation="vertical", 
            label="effect on cell entry",
            # low_high_ticks_only=True,
        )

    # Adjust scale bar tick labels
    ax_scale.set_yticks([-4, -2, 0])
    scale_labels = ["-4", "-2", "≥0"]
    ax_scale.set_yticklabels(labels=scale_labels)
    # Save scale bar
    if scale_bar_image != None:
        fig_scale.savefig(scale_bar_image)
    

    # Plot escape profiles and logo plots
    _, lineplot = dmslogo.draw_line(
        escape_df,
        x_col="site",
        height_col="site_escape",
        show_col="show_site",
        ax=line_plot,
        show_color="#CC6677",
        linewidth=0.5,
    )
    lineplot.set(ylabel=None, xlabel=None)
    lineplot.set_xlim(1,491)
    xticks = [100, 200, 300, 400]
    lineplot.set_xticks(xticks)
    x_labels = [
        "100", 
        "200",
        "300",
        "400",
    ]
    lineplot.set_xticklabels(labels=x_labels, rotation=90, horizontalalignment="center", fontsize=6)
    # Change all spines
    for axis in ["top", "bottom", "left", "right"]:
        lineplot.spines[axis].set_linewidth(1)
    lineplot.tick_params(axis="both", length=4, width=1, pad=1)
    

    _, logoplot = dmslogo.draw_logo(
        escape_df.query("show_site == True"),
        x_col="site",
        letter_col="mutant",
        letter_height_col="escape_median",
        ax=logo_plot,
        xtick_col="site",
        color_col="color",
        shade_color_col="shade_site",
        shade_alpha_col="shade_alpha",
        draw_line_at_zero="never",
    )

    logoplot.set(ylabel=None, xlabel=None)
    x_labels = logoplot.get_xticklabels()
    logoplot.set_xticklabels(labels=x_labels, rotation=90, horizontalalignment="center", fontsize=6)
    # Change all spines
    for axis in ["top", "bottom", "left", "right"]:
        logoplot.spines[axis].set_linewidth(1)
    logoplot.tick_params(axis="both", length=4, width=1, pad=1)


    # Set antibody specific y axis ticks
    if antibody_name == "2510C":
        # chart.set_ylim(-6.25, 56.25)
        yticks = [0, 25, 50]
        lineplot.set_yticks(yticks)
        lineplot.set_yticklabels(labels=["0", "25", "50"], fontsize=6)
        logoplot.set_yticks(yticks)
        logoplot.set_yticklabels(labels=["0", "25", "50"], fontsize=6)
        lineplot.set_title(
            antibody_name[0:2] + "." + antibody_name[2:], 
            fontsize=8,
            color="#44AA99",
        )
        logoplot.set_title(
            antibody_name[0:2] + "." + antibody_name[2:], 
            fontsize=8,
            color="#44AA99",
        )
    if antibody_name == "121F":
        # chart.set_ylim(-2.5, 22.5)
        yticks = [0, 10, 20]
        lineplot.set_yticks(yticks)
        lineplot.set_yticklabels(labels=["0", "10", "20"], fontsize=6)
        logoplot.set_yticks(yticks)
        logoplot.set_yticklabels(labels=["0", "10", "20"], fontsize=6)
        lineplot.set_title(
            antibody_name[0:2] + "." + antibody_name[2:], 
            fontsize=8,
            color="#999933",
        )
        logoplot.set_title(
            antibody_name[0:2] + "." + antibody_name[2:], 
            fontsize=8,
            color="#999933",
        )
    if antibody_name == "377H":
        # chart.set_ylim(-7.5, 67.5)
        yticks = [0, 30, 60]
        lineplot.set_yticks(yticks)
        lineplot.set_yticklabels(labels=["0", "30", "60"], fontsize=6)
        logoplot.set_yticks(yticks)
        logoplot.set_yticklabels(labels=["0", "30", "60"], fontsize=6)
        lineplot.set_title(
            antibody_name[0:2] + "." + antibody_name[2:], 
            fontsize=8,
            color="#AA4499",
        )
        logoplot.set_title(
            antibody_name[0:2] + "." + antibody_name[2:], 
            fontsize=8,
            color="#AA4499",
        )
    if antibody_name == "256A":
        # chart.set_ylim(-5, 45)
        yticks = [0, 20, 40]
        lineplot.set_yticks(yticks)
        lineplot.set_yticklabels(labels=["0", "20", "40"], fontsize=6)
        logoplot.set_yticks(yticks)
        logoplot.set_yticklabels(labels=["0", "20", "40"], fontsize=6)
        lineplot.set_title(
            antibody_name[0:2] + "." + antibody_name[2:], 
            fontsize=8,
            color="#AA4499",
        )
        logoplot.set_title(
            antibody_name[0:2] + "." + antibody_name[2:], 
            fontsize=8,
            color="#AA4499",
        )
    if antibody_name == "372D":
        # chart.set_ylim(-2.5, 22.5)
        yticks = [0, 10, 20]
        lineplot.set_yticks(yticks)
        lineplot.set_yticklabels(labels=["0", "10", "20"], fontsize=6)
        logoplot.set_yticks(yticks)
        logoplot.set_yticklabels(labels=["0", "10", "20"], fontsize=6)
        lineplot.set_title(
            antibody_name[0:2] + "." + antibody_name[2:], 
            fontsize=8,
            color="#AA4499",
        )
        logoplot.set_title(
            antibody_name[0:2] + "." + antibody_name[2:], 
            fontsize=8,
            color="#AA4499",
        )
    if antibody_name == "89F":
        # chart.set_ylim(-5, 45)
        yticks = [0, 20, 40]
        lineplot.set_yticks(yticks)
        lineplot.set_yticklabels(labels=["0", "20", "40"], fontsize=6)
        logoplot.set_yticks(yticks)
        logoplot.set_yticklabels(labels=["0", "20", "40"], fontsize=6)
        lineplot.set_title(
            antibody_name[0] + "." + antibody_name[1:],
            fontsize=8,
            color="#117733",
        )
        logoplot.set_title(
            antibody_name[0] + "." + antibody_name[1:],
            fontsize=8,
            color="#117733",
        )

In [ ]:
antibody_input_files = [
    escape_2510C,
    escape_121F,
    escape_377H,
    escape_256A,
    escape_372D,
    escape_89F,
]

antibody_output_files = [
    filtered_escape_2510C,
    filtered_escape_121F,
    filtered_escape_377H,
    filtered_escape_256A,
    filtered_escape_372D,
    filtered_escape_89F,
]

contacts_files = [
    contacts_2510C,
    contacts_121F,
    contacts_377H,
    contacts_256A,
    contacts_372D,
    contacts_89F,
]

scale_bar_images = [
    scale_bar_2510C,
    scale_bar_121F,
    scale_bar_377H,
    scale_bar_256A,
    scale_bar_372D,
    scale_bar_89F,
]

# Make output dir if doesn't exist
if not os.path.exists(out_dir):
    os.mkdir(out_dir)
if not os.path.exists(out_dir_images):
    os.mkdir(out_dir_images)

# Set figure size and subplots
fig, axes = plt.subplots(
    6, 
    2, 
    gridspec_kw={"width_ratios":[1,4]},
    figsize=(6, 6), 
    # sharex="col"
)
# Adjust spacing of subplots
fig.subplots_adjust(
    left=0.05, 
    right=1,
    bottom=0.075, 
    top=1, 
    wspace=0.1, 
    hspace=1,
)

# Iterate through list of antibody files
for i in range(len(antibody_input_files)):
    get_filtered_csv(
        antibody_input_files[i], 
        contacts_files[i],
        func_scores, 
        min_times_seen, 
        min_func_score, 
        n_selections, 
        frac_models,
        axes[i][0],
        axes[i][1],
        scale_bar_image = scale_bar_images[i],
        output_file = antibody_output_files[i],
    )

# Common X and Y axis labels
fig.text(0.5, 0, "site", ha="center", fontsize=8)
fig.text(0.0, 0.5, "site escape", va="center", rotation="vertical", fontsize=8)

# Save fig
fig.savefig(saved_image_path)

Create escape profile for 8.9F **single mutant** antibody validations

In [ ]:
# Set figure size and subplots
fig, axes = plt.subplots(
    1, 
    2, 
    gridspec_kw={"width_ratios":[2,1.5]},
    figsize=(3.5, 1), 
    # sharex="col"
)
# Adjust spacing of subplots
fig.subplots_adjust(
    left=0.05, 
    right=1,
    bottom=0.075, 
    top=1, 
    wspace=0.15, 
    hspace=1,
)

get_filtered_csv(
    escape_89F, 
    contacts_89F,
    func_scores, 
    min_times_seen, 
    min_func_score, 
    n_selections, 
    frac_models,
    axes[0],
    axes[1],
    sites = [89,119,125,126,129,135,138,148,149],
)

# Common X and Y axis labels
fig.text(0.5, -0.5, "site", ha="center", fontsize=8)
fig.text(-0.1, 0.5, "site escape", va="center", rotation="vertical", fontsize=8)

# Save fig
fig.savefig(validation_image_path)